In [2]:
import random
import sys
import os
import re

import requests
import pandas as pd
import numpy as np

import seaborn as sns
import spacy

from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline


# Used to filter out stop words in the text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wscot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1]:
# Take in a text file and save it as a string
# This is for Eye of the Worls Chapter 1
text_file = open(r"C:\Users\wscot\OneDrive\Desktop\DataSets\WheelOfTime\EoftWch_1.txt", "r", encoding = 'utf-8')
EoftW_ch_1 = text_file.read()
text_file.close()

In [14]:
# Take in a text file and save it as a string
# This is for wheel of time episode 1
text_file = open(r"C:\Users\wscot\OneDrive\Desktop\DataSets\WheelOfTime\WOT1.1.txt", "r")
WofT_1_1 = text_file.read()
text_file.close()

In [15]:
# Take in a text file and save it as a string
# This is for wheel of time episode 1
text_file = open(r"C:\Users\wscot\OneDrive\Desktop\DataSets\WheelOfTime\EoftWch_2.txt", "r")
EoftW_ch_2 = text_file.read()
text_file.close()

In [16]:
# cleaning the text
def clean_doc(text):
    #text = re.sub(r"[|]"", "", text)
    #text = text.replace('(|)', '', text)
    text = re.sub(r"\n|:", " ", text)
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    text = re.sub(r"  ", " ", text)
    text = text.lower().lstrip().rstrip()
    text = re.sub(r"  ", " ", text)
    text = re.sub(r"\b\d+\b", "", text)
   
    return text

In [17]:
# Tokenizing and removing stop words
def remove_stop_words(text):

    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
 
    filtered_sentence = []
 
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)

    return filtered_sentence


In [18]:
def listToString(s): 
    
    return ' '.join(s)

In [19]:
EoftW_ch_1_clean = clean_doc(EoftW_ch_1)
EoftW_ch_1_tokens = remove_stop_words(EoftW_ch_1_clean)
EoftW_ch_1_ready = listToString(EoftW_ch_1_tokens)

In [20]:
WofT_1_1_clean = clean_doc(WofT_1_1)
WofT_1_1_tokens = remove_stop_words(WofT_1_1_clean)
WofT_1_1_ready = listToString(WofT_1_1_tokens)

In [21]:
EoftW_ch_2_clean = clean_doc(EoftW_ch_2)
EoftW_ch_2_tokens = remove_stop_words(EoftW_ch_2_clean)
EoftW_ch_2_ready = listToString(EoftW_ch_2_tokens)

In [22]:
data = [EoftW_ch_1_ready, WofT_1_1_ready, EoftW_ch_2_ready]

In [23]:
#Call vectorizer

count_vectorizer = CountVectorizer()
vector_matrix = count_vectorizer.fit_transform(data)
vector_matrix

<3x2414 sparse matrix of type '<class 'numpy.int64'>'
	with 3406 stored elements in Compressed Sparse Row format>

In [24]:
tokens = count_vectorizer.get_feature_names()
tokens

['aah',
 'able',
 'abruptly',
 'accent',
 'accepted',
 'according',
 'across',
 'act',
 'actually',
 'adan',
 'add',
 'added',
 'admitting',
 'admonition',
 'adults',
 'aes',
 'affect',
 'affected',
 'affronted',
 'afield',
 'afraid',
 'age',
 'ages',
 'aggrieved',
 'aginor',
 'ago',
 'agonized',
 'agreed',
 'agreement',
 'ah',
 'ahead',
 'aim',
 'air',
 'alcaar',
 'aldonel',
 'ale',
 'alike',
 'alive',
 'almeara',
 'almost',
 'alone',
 'along',
 'alongside',
 'already',
 'alsbet',
 'althor',
 'althors',
 'altogether',
 'alvere',
 'alveres',
 'always',
 'among',
 'amusement',
 'anger',
 'angles',
 'angry',
 'anguished',
 'ankle',
 'announced',
 'another',
 'answered',
 'anticipating',
 'anybody',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'apologized',
 'apparently',
 'appeared',
 'appearing',
 'apple',
 'appreciate',
 'apprentice',
 'approach',
 'apron',
 'archery',
 'area',
 'arent',
 'arguing',
 'arm',
 'armor',
 'arms',
 'army',
 'aroma',
 'around',
 'arrived',
 'arrogance',


In [25]:
vector_matrix.toarray()

array([[0, 0, 3, ..., 6, 0, 4],
       [1, 3, 0, ..., 9, 0, 3],
       [0, 1, 2, ..., 0, 1, 0]], dtype=int64)

In [26]:
def create_dataframe(matrix, tokens):

    doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

In [27]:
create_dataframe(vector_matrix.toarray(),tokens)

,aah,able,abruptly,accent,accepted,according,across,act,actually,adan,...,yes,yet,youd,youll,young,younger,youngsters,youre,youths,youve
doc_1,0,0,3,0,1,1,1,1,3,0,...,1,2,0,1,5,1,1,6,0,4
doc_2,1,3,0,1,0,0,0,1,1,0,...,2,2,1,3,1,0,0,9,0,3
doc_3,0,1,2,0,0,0,2,0,0,1,...,0,1,0,1,3,2,0,0,1,0


In [28]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,[EoftW_ch_1_ready, WofT_1_1_ready,EoftW_ch_2_ready])

chapter empty road wheel time turns ages come pass leaving memories become legend legend fades myth even myth long forgotten age gave birth comes one age called third age age yet come age long past wind rose mountains mist wind beginning neither beginnings endings turning wheel time beginning born ever cloudcapped peaks gave mountains name wind blew east across sand hills shore great ocean breaking world flailed two rivers tangled forest called westwood beat two men walking cart horse rockstrewn track called quarry road spring come good month since wind carried icy chill would rather bear snow gusts plastered rand althors cloak back whipped earthcolored wool around legs streamed behind wished coat heavier worn extra shirt half time tried tug cloak back around caught quiver swinging hip trying hold cloak onehanded much good anyway bow arrow nocked ready draw particularly strong blast tugged cloak hand glanced father back shaggy brown mare felt little foolish wanting reassure tam still kind day wind howled rose aside quiet lay heavy land soft creak axle sounded loud comparison birds sang forest squirrels chittered branch expected really spring trees kept leaf needle winter green snarls last years bramble spread brown webs stone outcrops trees nettles numbered among weeds rest sorts sharp burrs thorns stinkweed left rank smell unwary boot crushed scattered white patches snow still dotted ground tight clumps trees kept deep shade sunlight reach held neither strength warmth pale sun sat trees east light crisply dark mixed shadow awkward morning made unpleasant thoughts without thinking touched nock arrow ready draw cheek one smooth movement way tam taught winter bad enough farms worse even oldest folk remembered must harsher still mountains number wolves driven two rivers guide wolves raided sheep pens chewed way barns get cattle horses bears sheep bear seen years longer safe dark men prey often sheep sun always tam taking steady strides side bela using spear walking staff ignoring wind made brown cloak flap like banner touched mares flank lightly remind keep moving thick chest broad face pillar reality morning like stone middle drifting dream sunroughened cheeks might lined hair sprinkling black among gray solidness though flood could wash around without uprooting feet stumped road impassively wolves bears well manner said things man kept sheep must aware best try stop tam althor getting emonds field guilty start rand returned watching side road tams matteroffactness reminding task head taller father taller anyone else district little tam physically except perhaps breadth shoulder gray eyes reddish tinge hair came mother tam said outlander rand remembered little aside smiling face though put flowers grave every year bel tine spring sunday summer two small casks tams apple brandy rested lurching cart eight larger barrels apple cider slightly hard winters curing tam delivered every year winespring inn use bel tine declared would take wolves cold wind stop spring even village weeks even tam traveled much days tam given word brandy cider even waited make delivery day festival keeping word important tam rand glad get away farm almost glad coming bel tine rand watched side road feeling grew watched tried shrug nothing moved made sound among trees except wind feeling persisted grew stronger hairs arms stirred skin prickled itched inside shifted bow irritably rub arms told stop letting fancies take nothing woods side road tam would spoken anything glanced shoulder blinked twenty spans back road cloaked figure horseback followed horse rider alike black dull ungleaming habit anything else kept walking backward alongside cart even looked riders cloak covered boot tops cowl tugged well forward part showed vaguely rand thought something odd horseman shadowed opening hood fascinated could see vaguest outlines face feeling looking right riders eyes could look away queasiness settled stomach shadow see hood felt hatred sharply could see snarling 

In [3]:
df_time_E1 = pd.read_csv (r"C:\Users\wscot\OneDrive\Desktop\DataSets\WheelOfTime\Episode_1_Location_by_Time.csv")

df_time_E1

,Time_in_Seconds,Primary_Location,Sub_Location,Unnamed: 3
0,57,Tarvalon,White Tower,NaN
1,198,Unknown,NaN,NaN
2,141,Two Rivers,NaN,NaN
3,57,Two Rivers,West Road,NaN
4,334,Two Rivers,Emmonds Field,WineSpring Inn
5,42,Two Rivers,Emmonds Field,Forge
6,27,Two Rivers,Emmonds Field,Streets
7,52,Two Rivers,Emmonds Field,Cauthon Home
8,313,Two Rivers,Emmonds Field,WineSpring Inn
9,21,Two Rivers,Emmonds Field,Streets


    I am going to also go through the actual text of those ten chapters and specifically mark the ammount of words in a specific location.
    Then I am goint to compare the time spent in each location from the book chapters to the text.
    After that, I can make visualizations explaining the differencec found between book and film.
